In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.fftpack import fft2, ifft2
from scipy.sparse import diags
from scipy.integrate import solve_ivp
from scipy.linalg import lu, solve_triangular

#A1
m = 64    # N value in x and y directions
n = m * m  # total size of matrix
dx = (10 - (-10)) / m

e0 = np.zeros((n, 1))  # vector of zeros
e1 = np.ones((n, 1))   # vector of ones
e2 = np.copy(e1)    # copy the one vector
e4 = np.copy(e0)    # copy the zero vector

for j in range(1, m+1):
    e2[m*j-1] = 0  # overwrite every m^th value with zero
    e4[m*j-1] = 1  # overwirte every m^th value with one

# Shift to correct positions
e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]

e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]

# Place diagonal elements
diagonals = [e1.flatten(), e1.flatten(), e5.flatten(), 
             e2.flatten(), -4 * e1.flatten(), e3.flatten(), 
             e4.flatten(), e1.flatten(), e1.flatten()]
offsets = [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)]

A = spdiags(diagonals, offsets, n, n).toarray()/dx**2
A[0,0]=2/dx**2
print("A:" , A)

#A2 
diagonals = [e1.flatten(), -e1.flatten(), e1.flatten(), -e1.flatten()]
offsets = [-(n-m), -m, m, (n-m)]
B = spdiags(diagonals, offsets, n, n).toarray()/(dx*2)
print(B)
#A3
e3 = np.ones((n, 1))
for i in range(0, n, m):
    e3[i] = 0 
    e0[i] = 1

e4 = np.zeros((n, 1))
for i in range(m - 1, n, m):
    e4[i] = 1  


diagonals = [e0.flatten(), -e2.flatten(), e3.flatten(), -e4.flatten()]
offsets = [-m + 1, -1, 1, m - 1]

C = spdiags(diagonals, offsets, n, n).toarray()/(dx*2)
print("C:",C)


# Domain setup
Lx, Ly = 20, 20  
Nx, Ny = 64, 64  
N = Nx * Ny
x2 = np.linspace(-Lx/2, Lx/2, Nx + 1)
x = x2[:Nx]
y2 = np.linspace(-Ly/2, Ly/2, Ny + 1)
y = y2[:Ny]
dx, dy = x[1] - x[0], y[1] - y[0]
X, Y = np.meshgrid(x, y)


kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, Nx/2), np.arange(-Nx/2, 0)))
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, Nx/2), np.arange(-Nx/2, 0)))
kx[0] = ky[0] = 1e-6 
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2
K[0, 0] = 1e-6  # Avoid division by zero

# Initial condition for vorticity
omega0 = np.exp(-X**2 - Y**2 / 20)  # Gaussian vorticity
omega = omega0.reshape(N)

nu = 0.001  
tspan = np.arange(0, 4 +0.5, 0.5)


def spc_rhs(t, omega, Nx, Ny, N, A, B, C, K, nu):
    
    w = omega.reshape((Ny, Nx))
    wt = fft2(w)
    psit = -wt / K  
    psi = np.real(ifft2(psit))
    
    rhs = (
        nu * np.dot(A, omega)  
        + (np.dot(B, omega)) * (np.dot(C, psi.ravel()))  
        - (np.dot(B, psi.ravel())) * (np.dot(C, omega))
    )
    return rhs

# Solve the ODE and plot the results
sol = solve_ivp(
    spc_rhs,
    [0, 4],  
    omega,  
    t_eval=tspan,  
    args=(Nx, Ny, N, A, B, C, K, nu), 
    method='RK45'
)

A1= sol.y
print("A1:",A1)

def GE_rhs(t,w2, Nx, Ny, N, A, B, C, K, nu):
    psi= np.linalg.solve(A,w2)
    rhs=nu*np.dot(A,w2)+(np.dot(B,w2))*(np.dot(C,psi))-(np.dot(B,psi))*(np.dot(C,w2))
    return rhs

# Solve the ODE and plot the results
sol = solve_ivp(GE_rhs, [0, 4], omega, t_eval=tspan, args=(Nx, Ny, N, A, B, C, K, nu), method='RK45')
A2 = sol.y
print("A2:",A2)



P,L,U=lu(A)

def LU_rhs(t,w2, Nx, Ny, N, A, B, C, K,nu):
    Pb=np.dot(P,w2)
    y=solve_triangular(L,Pb,lower=True)
    psi=solve_triangular(U,y)
    rhs=nu*np.dot(A,w2)+(np.dot(B,w2))*(np.dot(C,psi))-(np.dot(B,psi))*(np.dot(C,w2))
    return rhs

# Solve the ODE and plot the results
sol = solve_ivp(LU_rhs, [0, 4], omega, t_eval=tspan, args=(Nx, Ny, N, A, B, C, K, nu), method='RK45')
A3 = sol.y
print("A3:",A3)


A: [[ 20.48  10.24   0.   ...   0.     0.     0.  ]
 [ 10.24 -40.96  10.24 ...   0.     0.     0.  ]
 [  0.    10.24 -40.96 ...   0.     0.     0.  ]
 ...
 [  0.     0.     0.   ... -40.96  10.24   0.  ]
 [  0.     0.     0.   ...  10.24 -40.96  10.24]
 [  0.     0.     0.   ...   0.    10.24 -40.96]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
C: [[ 0.   1.6  0.  ...  0.   0.   0. ]
 [-1.6  0.   1.6 ...  0.   0.   0. ]
 [ 0.  -1.6  0.  ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ...  0.   1.6  0. ]
 [ 0.   0.   0.  ... -1.6  0.   1.6]
 [ 0.   0.   0.  ...  0.  -1.6  0. ]]
A1 [[2.50656748e-46 3.60757664e-45 1.77528687e-44 ... 1.87383205e-42
  4.81529623e-42 1.14459549e-41]
 [1.17762859e-43 6.54982432e-43 2.56837085e-42 ... 1.57584298e-40
  3.55640184e-40 7.45697326e-40]
 [4.55107657e-41 1.93910312e-40 6.40898000e-40 ... 2.48209098e-38
  5.07713545e-38 9.71169361e-38]
 ...
 [1.9